In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lid_driven_cavity import operators, states
import numpy as np

import scipy as sp
np.set_printoptions(linewidth=2000, threshold=5000, precision=6)

In [3]:
# Physics parameters
reynolds_numbers = [100, 400, 1000, 3200, 5000]

# Boundary conditions
u_top = (1, 0)
u_bot = (0, 0)
u_left = (0, 0)
u_right = (0, 0)

# Spatial parameters
Lx = 1.0
Ly = 1.0

In [4]:
# Physics parameter
Re = reynolds_numbers[-1]

# Spatial resolution
N = 3  # number of cells in each direction
h = 1.0 / N

# Temporal resolution
k = 1.0  # FIXME arbitrary for now, likely will need to decrease

# Setup

1. Assume $\phi^n$ and $u^n$ are known at all interior, boundary, and ghost points.

In [5]:
phi_n = states.State(np.zeros((N+2, N+2)), Nx=N+2, Ny=N+2)
u_n = states.State(np.zeros((N+2, N+1)), Nx=N+1, Ny=N+2)
v_n = states.State(np.zeros((N+1, N+2)), Nx=N+2, Ny=N+1)
print(phi_n)
print(u_n)
print(v_n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


2. Assume $G(\phi^n)$ is known at all interior, boundary, and ghost points.

In [6]:
Gx_phi_n = operators.compute_gradient_x(phi_n)  # on phi grid, no left boundary
Gy_phi_n = operators.compute_gradient_y(phi_n)  # on phi grid, no bottom boundary
print(Gx_phi_n)
print(Gy_phi_n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


3. Assume $N(u^{n−1})$ is known at all interior points.

In [7]:
Nu_nm1 = states.State(np.zeros((N+2, N-1+2)), Nx=N-1+2, Ny=N+2)
Nv_nm1 = states.State(np.zeros((N-1+2, N+2)), Nx=N+2, Ny=N-1+2)
print(Nu_nm1)
print(Nv_nm1)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


# Prediction step

4. Compute $\hat{u}^n$ at all interior points.

In [8]:
u_hat_n = operators.compute_interpolated_x(u_n)  # on phi grid, no boundary columns
v_hat_n = operators.compute_interpolated_y(v_n)  # on phi grid, no boundary rows
print(u_hat_n)
print(v_hat_n)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


5. Compute $N(u^n)$ at all interior points.

In [9]:
Nu = operators.compute_Nu(u_n, v_hat_n, h)
Nv = operators.compute_Nv(v_n, u_hat_n, h)
print(Nu)
print(Nv)

[[ 0.  0.  0.  0.]
 [ 0. -0. -0.  0.]
 [ 0. -0. -0.  0.]
 [ 0. -0. -0.  0.]
 [ 0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.]
 [ 0. -0. -0. -0.  0.]
 [ 0. -0. -0. -0.  0.]
 [ 0.  0.  0.  0.  0.]]


6. Evaluate $g(x, t_{n+1})$ at all required locations on the domain boundary.

7. Solve for $u^*$ at all interior, boundary, and ghost points.

In [10]:
Lu = operators.compute_laplace(u_n)
Lv = operators.compute_laplace(v_n)
print(Lu)
print(Lv)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [11]:
# Assemble RHS
b_interior_u = (k/2) * (3*Nu.get_matrix()-Nu_nm1.get_matrix()) + u_n.get_matrix() + (k/2)*(1/Re)*Lu.get_matrix()
b_interior_u = states.strip_boundaries(b_interior_u)

# pad with zeros on boundaries
b_u = np.zeros((b_interior_u.shape[0]+2, b_interior_u.shape[1]+2))
b_u[1:-1, 1:-1] = b_interior_u

# convert to vector
b_u = states.matrix_to_vector(b_u)

# Assemble operator
laplacian, b = operators.assemble_laplacian_operator_u(b_u, Nx=N+1, Ny=N+2, h=h)

# Assemble linear system
A1 = sp.sparse.eye(laplacian.shape[0], dtype=float, format='csr')
A2 = (k/2)*(1/Re)*laplacian.get_csr()
A = A1 + A2

print(A.toarray())
print(b)

# Solve implicit step
u_star = sp.sparse.linalg.spsolve(A, b)
print(states.vector_to_matrix(u_star, 4, 5))

[[1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e-04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e-04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e

In [13]:
# Assemble RHS
b_interior_v = (k/2) * (3*Nv.get_matrix()-Nv_nm1.get_matrix()) + v_n.get_matrix() + (k/2)*(1/Re)*Lv.get_matrix()
b_interior_v = states.strip_boundaries(b_interior_v)

# pad with zeros on boundaries
b_v = np.zeros((b_interior_v.shape[0]+2, b_interior_v.shape[1]+2))
b_v[1:-1, 1:-1] = b_interior_v

# convert to vector
b_v = states.matrix_to_vector(b_v)

# Assemble operator
laplacian, b = operators.assemble_laplacian_operator_v(b_v, Nx=N+2, Ny=N+1, h=h)

# Assemble linear system
A1 = sp.sparse.eye(laplacian.shape[0], dtype=float, format='csr')
A2 = (k/2)*(1/Re)*laplacian.get_csr()
A = A1 + A2

print(A.toarray())
print(b)

# Solve implicit step
v_star = sp.sparse.linalg.spsolve(A, b)
print(states.vector_to_matrix(v_star, 4, 5))

TypeError: assemble_laplacian_operator_v() missing 1 required positional argument: 'k'

# Poisson problem

8. Compute $D(u^*)$ at all interior points.

In [ ]:
u_star_matrix = states.get_matrix(u_star, 4, 5)
v_star_matrix = states.get_matrix(v_star, 5, 4)
print(u_star_matrix)
print(v_star_matrix)
dudx = u_star_matrix[:, 1:] - u_star_matrix[:, :-1]
dvdy = v_star_matrix[1:, :] - v_star_matrix[:-1, :]
print(dudx)
print(dvdy)
dudx_interior = dudx[1:-1, :]
dvdy_interior = dvdy[:, 1:-1]
print(dudx_interior)
print(dvdy_interior)
Dstar = (dudx_interior + dvdy_interior) / h
print(Dstar)

9. Solve for $\phi^{n+1}$ at all interior and ghost points.

In [ ]:
b_interior = Dstar / k

# pad with zeros
b = np.zeros((b_interior.shape[0]+2, b_interior.shape[1]+2))
b[1:-1, 1:-1] = b_interior

# convert to vector
b = states.get_vector(b)

# TODO add ones on main diagonal for state_ij = 0 entries!!!
Lphi, b = operators.assemble_laplacian_operator_phi(b, u_star_matrix, v_star_matrix, Nx=N+2, Ny=N+2, h=h, k=k)
print(Lphi.get_csr().toarray())

# Assemble linear system
A = Lphi.get_csr()
# A = (k/2)*(1/Re)*laplacian.get_csr()

phi_np1 = sp.sparse.linalg.spsolve(A, b)
print(phi_np1)

# Correction step

10. Compute $G(\phi^{n+1})$ at all interior, boundary, and ghost points.

In [ ]:
# G_phi_np1 = lid.gradient(phi_np1)

11. Correct $u^*$ to obtain $u^{n+1}$ at all interior, boundary, and ghost points.

In [ ]:
# u_np1 = lid.take_correction_step(u_star, G_phi_np1)

# Bookkeeping

12. Update counter from $n$ to $n + 1$ and time from $t_n$ to $t_{n+1}$.

In [ ]:
# n = n + 1
# tn = tn + k

13. Repeat from Step 1.